In [ ]:
%pip install openai praw

In [12]:
import openai

In [11]:
import praw


In [19]:
import os


In [20]:
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent='sentiment analysis test')

In [22]:
print('CLIENT_ID: ', client_id)

CLIENT_ID:  ettruFw36oNOPefU9Ah0Nw


In [23]:
for submission in reddit.subreddit("finance").hot(limit=5):
    print(submission.title)

Moronic Monday - June 06, 2023 - Your Weekly Questions Thread
We are Wall Street Journal reporters covering the collapse of SVB. Ask us anything.
why universities don't teach technical analysis?
The junkiest junk bonds feel the pain of economic decline — Lowest-rated company debt slumped by the most in eight months
Column: US money supply falling at fastest rate since 1930s
